In [ ]:
# 1. Development Magic (Keep this! It's essential for VS Code syncing)
%load_ext autoreload
%autoreload 2

# 2. Quiet Installation
# The -q flag makes it "quiet" so you don't get a wall of text every time
%pip install -q -e "../hypencoder-paper"

In [ ]:
# %pip install transformers tqdm more_itertools scikit-learn torch datasets fire omegaconf jsonlines

In [ ]:
# best practice would probably be to create my own fork and clone from there (not currently doing this)
#!git clone https://github.com/Steve-Falkovsky/Hypencoder-Entity-Linking.git

In [ ]:
#%pip install -e /content/hypencoder-paper

In [ ]:
# change directory (helps when running the scripts)
%cd ../hypencoder-paper

In [ ]:
# loading the data
from datasets import load_dataset

dataset = load_dataset("Stevenf232/hypencoder_contrastiveLoss_nameOnly")
train_data = dataset['train']
val_data = dataset['validation']

In [ ]:
# saving the data to a file
train_data.to_json('data/train.jsonl', lines=True)
val_data.to_json('data/val.jsonl', lines=True)

In [ ]:
# tokenizing the data before training

# training
!python hypencoder_cb/utils/tokenizer_utils.py \
--standard_format_jsonl='data/train.jsonl' \
--output_file='data/train_tokenized.jsonl' \
--tokenizer="cambridgeltl/SapBERT-from-PubMedBERT-fulltext" \
--add_special_tokens=True \
--query_max_length=32 \
--item_max_length=512

# validation
!python hypencoder_cb/utils/tokenizer_utils.py \
--standard_format_jsonl='data/val.jsonl' \
--output_file='data/val_tokenized.jsonl' \
--tokenizer="cambridgeltl/SapBERT-from-PubMedBERT-fulltext" \
--add_special_tokens=True \
--query_max_length=32 \
--item_max_length=512

Everything in the output above is [00:00] which seems quite suspicious! (or it could just be really fast)



---



Training the hypencoder

In [ ]:
import fire
!python hypencoder_cb/train/train.py hypencoder_cb/train/configs/hypencoder.2_layer_SapBERT.yaml

In [ ]:
%pip install huggingface_hub

In [ ]:
# push the model to HuggingFace
# the model was saved in the model directory

from huggingface_hub import upload_folder

# Path where your model files (weights, config, etc.) are saved in Colab
local_folder_path = "/content/hypencoder-paper/model/hypencoder.2_layer_SapBERT/checkpoint-480"

# Your desired repository ID on Hugging Face (e.g., "your-username/my-generic-model")
repo_id = "Stevenf232/SapBERT_hypencoder_freeze"

# You may need to create the repository first if it doesn't exist
from huggingface_hub import create_repo
create_repo(repo_id, exist_ok=True)

upload_folder(
    folder_path=local_folder_path,
    repo_id=repo_id,
    repo_type="model", # or "dataset" or "space"
    commit_message="Upload trained model from Colab"
)